In [323]:
%reload_ext black
from bs4 import BeautifulSoup
from bs4 import NavigableString
import pandas as pd
import pprint

import urllib

In [324]:
news_url = 'https://mars.nasa.gov/news/'
img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

# Latest Mars Articles

In [327]:
#open and parse the page
page = urllib.request.urlopen(news_url)
news_soup = BeautifulSoup(page, 'html.parser')

#zip title and description items for iterability
zipped = zip(
    news_soup.find_all('div', class_='content_title'),
    news_soup.find_all('div', class_='image_and_description_container')
            )

#loop though zip object and extract data
articles = []
for div in zipped:
    info_dict = {
    'article_title' : div[0].a.text.strip(),
    #'date_posted' : div[2].text.strip(),
    'description_text' : div[1].text.strip(),
    'article_link' : div[1].a['href'].strip(),
    }
    articles.append(info_dict)

##grab the latest article
latest_article_title = articles[0]['article_title']
latest_article_desc = articles[0]['description_text']

print(latest_article_title)
print(latest_article_desc)

Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover
NASA chose a seventh-grader from Virginia as winner of the agency's "Name the Rover" essay contest. Alexander Mather's entry for "Perseverance" was voted tops among 28,000 entries.


# Mars Images from JPL

In [328]:
#open and parse the page
page = urllib.request.urlopen(img_url)
img_soup = BeautifulSoup(page, 'html.parser')



Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover
NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network
NASA Administrator Statement on Moon to Mars Initiative, FY 2021 Budget
NASA's Mars 2020 Rover Closer to Getting Its Name
NASA Invites Students to Name Mars 2020 Rover
NASA's Curiosity Mars Rover Finds a Clay Cache
